In [1]:
import findspark
findspark.init()

import pyspark
import random

print('init')

sc = pyspark.SparkContext(appName="Pi")
num_samples = 10000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()
print(sc)

pi = 4 * count / num_samples
print(pi)

sc.stop()


init
<SparkContext master=local[*] appName=Pi>
3.1492


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark

In [6]:
# spark is an existing SparkSession
df = spark.read.csv("../nlp/stanfordSentimentTreebank/dictionary_sm.txt")
# Displays the content of the DataFrame to stdout
df.show()

df2 = spark.read.csv("../nlp/stanfordSentimentTreebank/sentiment_labels_sm.txt")
df2.show()


+--------------------+
|                 _c0|
+--------------------+
|             text|id|
|                 !|0|
|           ! '|22935|
|          ! ''|18235|
|       ! Alas|179257|
|   ! Brilliant|22936|
| ! Brilliant !|40532|
|! Brilliant ! '|2...|
|       ! C'mon|60624|
|! Gollum 's ` per...|
|               ! Oh |
|  ! Romething|140882|
|        ! Run|179259|
|   ! The Movie|60625|
|! The camera twir...|
|! True Hollywood ...|
|        ! Wow|179261|
|     ! Zoom !|179262|
|           !?|220445|
|         !? '|220446|
+--------------------+
only showing top 20 rows

+------------+
|         _c0|
+------------+
|id|sentiment|
|       0|0.5|
|       1|0.5|
|   2|0.44444|
|       3|0.5|
|   4|0.42708|
|     5|0.375|
|   6|0.41667|
|   7|0.54167|
|   8|0.33333|
|   9|0.45833|
|  10|0.47222|
|  11|0.59722|
|  12|0.33333|
|  13|0.93056|
|  14|0.80556|
|  15|0.81944|
|  16|0.76389|
|      17|0.5|
|      18|0.5|
+------------+
only showing top 20 rows



In [18]:
df = spark.read.load("../nlp/stanfordSentimentTreebank/dictionary_sm.txt",
                     format="csv", sep="|", inferSchema="true", header="true")
df.show()


+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|                   #| 60626|
+--------------------+------+
only showing top 20 rows



In [14]:
df.printSchema()
#df.createGlobalTempView("sentences")
spark.sql("SELECT * FROM global_temp.sentences").show()


root
 |-- _c0: string (nullable = true)

+--------------------+
|                 _c0|
+--------------------+
|             text|id|
|                 !|0|
|           ! '|22935|
|          ! ''|18235|
|       ! Alas|179257|
|   ! Brilliant|22936|
| ! Brilliant !|40532|
|! Brilliant ! '|2...|
|       ! C'mon|60624|
|! Gollum 's ` per...|
|               ! Oh |
|  ! Romething|140882|
|        ! Run|179259|
|   ! The Movie|60625|
|! The camera twir...|
|! True Hollywood ...|
|        ! Wow|179261|
|     ! Zoom !|179262|
|           !?|220445|
|         !? '|220446|
+--------------------+
only showing top 20 rows



In [17]:
from pyspark.sql.types import *

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("../nlp/stanfordSentimentTreebank/dictionary_sm.txt")
parts = lines.map(lambda l: l.split("|"))
# Each line is converted to a tuple.
wordid = parts.map(lambda p: (p[0].strip(), p[1].strip()))

# The schema is encoded in a string.
schemaString = "word id"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaWord = spark.createDataFrame(wordid, schema)

# Creates a temporary view using the DataFrame
schemaWord.createOrReplaceTempView("word")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT * FROM word")

results.show()


+--------------------+------+
|                word|    id|
+--------------------+------+
|                text|    id|
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
+--------------------+------+
only showing top 20 rows



In [27]:
#df.write.saveAsTable("saved_words")

spark.sql("show databases").show()
spark.sql("show tables").show()
spark.sql("select * from saved_words").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
| default|saved_words|      false|
|        |       word|       true|
+--------+-----------+-----------+

+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|                   #| 60626|
+--------------------+-

In [33]:
df.write.mode("overwrite").parquet("words.parquet")
parquetFile = spark.read.parquet("words.parquet")
parquetFile.createOrReplaceTempView("parquetFile")
words = spark.sql("SELECT * FROM parquetFile ")
words.show()

+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|                   #| 60626|
+--------------------+------+
only showing top 20 rows



In [37]:
from pyspark.sql import Row
sc = spark.sparkContext

squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                  .map(lambda i: Row(single=i, double=i ** 2)))
squaresDF.write.parquet("data/test_table/key=1")

# Create another DataFrame in a new partition directory,
# adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                .map(lambda i: Row(single=i, triple=i ** 3)))
cubesDF.write.parquet("data/test_table/key=2")


AnalysisException: 'path file:/C:/Users/dougc/git/machineLearning/notebooks/data/test_table/key=1 already exists.;'

In [38]:

# Read the partitioned table
mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDF.printSchema()
mergedDF.show()

#

root
 |-- double: long (nullable = true)
 |-- single: long (nullable = true)
 |-- triple: long (nullable = true)
 |-- key: integer (nullable = true)

+------+------+------+---+
|double|single|triple|key|
+------+------+------+---+
|  null|     9|   729|  2|
|  null|    10|  1000|  2|
|    16|     4|  null|  1|
|    25|     5|  null|  1|
|  null|     6|   216|  2|
|  null|     7|   343|  2|
|  null|     8|   512|  2|
|     1|     1|  null|  1|
|     4|     2|  null|  1|
|     9|     3|  null|  1|
+------+------+------+---+



In [44]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Generate a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()
result_pdf.head()

,0,1,2
0,0.500977,0.823422,0.159175
1,0.855146,0.959210,0.589888
2,0.656151,0.502322,0.076855
3,0.943564,0.917697,0.778736
4,0.084412,0.848397,0.278503


In [51]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print(distData.reduce(lambda a, b: a + b))
print(distData.map(lambda a: a ** 2).reduce(lambda a,b: a+b))


15
55


In [57]:
distFile = sc.textFile("../nlp/stanfordSentimentTreebank/dictionary*.txt")
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)


11774506

In [1]:
# from pyspark import SparkContext, SparkConf
# conf = SparkConf()
# conf.setMaster('spark://192.168.1.28:7077')
# conf.setAppName('mynewapp2')
# sc2 = SparkContext(conf=conf)
# print (sc2)

# def mod(x):
#     import numpy as np
#     return (x, np.mod(x, 2))
# rdd = sc2.parallelize(range(100)).map(mod).take(10)
# rdd

<SparkContext master=spark://192.168.1.28:7077 appName=mynewapp2>


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, 192.168.1.28, executor 0): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:109)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:346)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:310)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Tas